In [1]:
# Input file name definitions
fnameInput = 'MEMMAL_o4a' # model file name = USER input

# Names for the updated input files
fileComps = 'Compartments_MM.txt' 
fileSpecies = 'Species_MM.txt' 
fileRatelaws = 'RatelawsNoSM_MM.txt' 
fileOmicsdata = 'OmicsData_MM.txt' 
fileGeneReg = 'GeneReg_MM.txt'
fileParamsOut = 'ParamsAll_MM.txt' # Output file to store all parameter (rate constants) values

In [2]:
import sys
import os
sys.path.append(os.getcwd()[0:os.getcwd().rfind('/')]+'/bin')

import libsbml
import importlib
import amici
import numpy as np
import re
import pandas as pd
from antimony import *
from modules.copyDir import copyDirectory

# Optional packages to import
import amici.plotting
import matplotlib.pyplot as plt

In [4]:
fileModel = open(fnameInput+'.txt','w') 
fileModel.write("# MEchanistic Modeling with MAchine Learning (MEMMAL) model (One4All version) by Cemal Erdem, Ph.D. (September 2022) \n")
fileModel.write("model "+fnameInput+"()\n") # model name

# SBML file name
sbml_file = fnameInput+'.xml'
# Name of the model that will also be the name of the python module
model_name = sbml_file[0:-4] 
# Directory to which the generated model code is written
model_output_dir = model_name 
# The AMICI package will create this folder while compiling the model, refer to AMICI github page for more details

In [ ]:
# Initializing compartment and volume lists
compartments = []
volumes = []

# Create/write compartments
compartment_sheet = np.array([np.array(line.strip().split("\t")) for line in open(fileComps)])

#read in each line minus the header row of compartments file
for row in compartment_sheet[1:]:
    compartments.append(row[0])
    volumes.append(row[1])
    
fileModel.write("\n  # Compartments and Species:\n") # Antimony Compartments/Species module title
for idx in range(len(compartments)):
    compName = compartments[idx]
    fileModel.write("  Compartment %s; " % (compName))
fileModel.write("\n")

In [ ]:
model_genes = list(pd.read_csv(fileOmicsdata,sep='\t',index_col=0,header=0).index)
species_sheet = np.array([np.array(line.strip().split("\t")) for line in open(fileSpecies, encoding='latin-1')])
# species_sheet = pd.read_csv(fileSpecies,header=0,index_col=0,sep="\t",encoding='latin-1')
mRNAnames = [i for i in [row[0] for row in species_sheet] if i.startswith('m_')]
# sp_mrna = species_sheet.index.map(lambda s: s.startswith('m_')) # Separate mRNA species from protein/complexes

### Write species and assign compartments
#read in each line minus the header row of species file
species_compartments = [] # Create a list of "home" compartments for each species in the model
for row in species_sheet[1:]:
    species_compartments.append(row[1]) 
species_compartments = np.array(species_compartments)

#Write each species to model txt file
fileModel.write("\n")
for idx,val in enumerate(species_sheet[1:]):
    fileModel.write("  Species ")
    fileModel.write("%s in %s" % (val[0], species_compartments[idx]))
    fileModel.write(";\n")
    
omicsRead = pd.read_csv(fileOmicsdata,header=0,index_col=0,sep="\t")
gExp_mpc = np.float64(omicsRead.values[:,0])
mExp_mpc = np.float64(omicsRead.values[:,1])
kGin = np.float64(omicsRead.values[:,2])
kGac = np.float64(omicsRead.values[:,3])
kTCleak = np.float64(omicsRead.values[:,4])
kTCmaxs = np.float64(omicsRead.values[:,5])
kTCd = np.float64(omicsRead.values[:,6])
gnames = list(omicsRead.index.values)
ac_genenames = ['ag_' + x for x in gnames]
ia_genenames = ['ig_' + x for x in gnames]

Vn = np.double(compartment_sheet[3,1])
Vc = np.double(compartment_sheet[1,1])
mpc2nMVc = (1E9/(Vc*6.023E+23))

genesComp = np.array(omicsRead.values[:,12])
# CC mRNA IC exceptions:
cc_mrna = pd.read_csv('Initializer.txt',sep='\t',squeeze=True,usecols=['Step1_mrna','Step1_mrna_mpc'],index_col='Step1_mrna')
cc_mrna = cc_mrna[cc_mrna.index.notnull()]

for c in range(len(cc_mrna)):
    cmrna_sp = 'm_'+str(cc_mrna.index[c])
    mExp_mpc[list(mRNAnames).index(cmrna_sp)] = cc_mrna[c]

for idx, val in enumerate(ac_genenames):
    fileModel.write("  Species ")
    fileModel.write("%s in %s" % (val, genesComp[idx]))
    fileModel.write(";\n")
for idx, val in enumerate(ia_genenames):
    fileModel.write("  Species ")
    fileModel.write("%s in %s" % (val, genesComp[idx]))
    fileModel.write(";\n")   

# Write reactions
fileModel.write("\n\n  # Reactions:\n") # Antimony Reactions module title

# reads in file from excel and gets rid of first row and column (they're data labels)
ratelaw_sheet = np.array([np.array(line.strip().split("\t")) for line in open(fileRatelaws)])
ratelaw_data = np.array([line[1:] for line in ratelaw_sheet[1:]])
ratelaw_rows = np.array([line[0] for line in ratelaw_sheet[1:]])

# builds the important ratelaw+stoic lines into the txt file 
paramnames = []
paramvals = []
paramrxns = []
paramidxs = []
for rowNum, ratelaw in enumerate(ratelaw_data):
    reactants = []
    products = []
    formula = "k"+str(rowNum+1)+"*"
    if ";" in ratelaw[1]: # Check if non-empty
        spRPs = ratelaw[1].split(";") # separate into reactants and products
        reacts = spRPs[0].split("+") # separate into individual reactants, if any
        for rr in reacts:
            if "," in rr.strip():
                pp = rr.split(",")
                for j in range(0,int(pp[1])):
                    reactants.append(pp[0].strip())
                    formula = formula+pp[0]+"*"
            else:
                if not rr.isspace():
                    reactants.append(rr.strip())
                    formula = formula+rr+"*"
        prdcts = spRPs[1].split("+") # separate into individual products, if any
        for rr in prdcts:
            if "," in rr.strip():
                pp = rr.split(",")
                for j in range(0,int(pp[1])):
                    products.append(pp[0].strip())
            else:
                if not rr.isspace():
                    products.append(rr.strip())
    if "k" not in ratelaw[2]:
        # the mass-action formula
        formula = formula[:-1]
        #the parameter
        paramnames.append("k"+str(rowNum+1))
        paramvals.append(np.double(ratelaw[2]))
        paramrxns.append(ratelaw_sheet[rowNum+1][0])
        paramidxs.append(int(0))
    else:
        # specific formula (non-mass-action)
        formula = ratelaw[2]
        j = 1
        params = np.genfromtxt(ratelaw[3:], float) # parameters
        params = params[~np.isnan(params)]
        if len(params) == 1:
            paramnames.append("k"+str(rowNum+1)+"_"+str(j))
            paramvals.append(float(ratelaw[j+2]))
            paramrxns.append(ratelaw_sheet[rowNum+1][0])
            paramidxs.append(int(0))
            pattern = 'k\D*\d*'
            compiled = re.compile(pattern)
            matches = compiled.finditer(formula)
            for ematch in matches:
                formula = formula.replace(ematch.group(),paramnames[-1])
        else:
            for q,p in enumerate(params):
                paramnames.append("k"+str(rowNum+1)+"_"+str(j))
                paramvals.append(float(ratelaw[j+2]))
                paramrxns.append(ratelaw_sheet[rowNum+1][0])
                paramidxs.append(q)
                pattern1 = 'k(\D*)\d*'+'_'+str(j)
                compiled1 = re.compile(pattern1)
                matches1 = compiled1.finditer(formula)
                for ematch in matches1:
                    formula = formula.replace(ematch.group(),paramnames[-1])
                j +=1
    #don't include reactions without products or reactants
    if products == [] and reactants == []:
        pass
    else:
        fileModel.write("  %s: %s => %s; (%s)*%s;\n" % (ratelaw_rows[rowNum], " + ".join(reactants), " + ".join(products), formula, ratelaw[0]))

rowLastNo = rowNum+1
for rowNum, val in enumerate(ia_genenames):

    formula = "k"+str(rowLastNo+rowNum+1)+"*"+val
    fileModel.write("  %s: %s => %s; (%s);\n" % ('vGac'+str(rowNum+1), val, ac_genenames[rowNum], formula))
    
    paramnames.append("k"+str(rowLastNo+rowNum+1))
    paramvals.append(kGac[rowNum])

    paramrxns.append('vGac'+str(rowNum+1))
    paramidxs.append(0)

rowLastNo = rowLastNo+rowNum+1
for rowNum, val in enumerate(ac_genenames):

    formula = "k"+str(rowLastNo+rowNum+1)+"*"+val
    fileModel.write("  %s: %s => %s; (%s);\n" % ('vGin'+str(rowNum+1), val, ia_genenames[rowNum], formula))
    
    paramnames.append("k"+str(rowLastNo+rowNum+1))
    paramvals.append(kGin[rowNum])

    paramrxns.append('vGin'+str(rowNum+1))
    paramidxs.append(0)
    
TARsRead = pd.read_csv(fileGeneReg,header=0,index_col=0,sep="\t")
TARs0 = (TARsRead.values)
numberofTARs = len(TARsRead.columns)
numberofgenes = int(len(gExp_mpc))

tcnas = np.ones((numberofgenes, numberofTARs))
tck50as = np.zeros((numberofgenes, numberofTARs))
tcnrs = np.ones((numberofgenes, numberofTARs))
tck50rs = np.zeros((numberofgenes, numberofTARs))
for qq in range(numberofgenes):
    for ww in range(numberofTARs):
        pars = str(TARs0[qq,ww]).find(';')
        if pars>0:
            nH = np.float(TARs0[qq,ww][0:pars])
            kH = np.float(TARs0[qq,ww][pars+2::])
            if nH>0:
                tcnas[qq,ww] = nH
                tck50as[qq,ww] = kH
            else:
                tcnrs[qq,ww] = abs(nH)
                tck50rs[qq,ww] = kH
mpc2nmcf_Vn = 1.0E9/(np.float(Vn)*6.023E+23)
mpc2nmcf_Vc = 1.0E9/(np.float(Vc)*6.023E+23)

mrnas2mod = ["m_CCND1","m_CCND2","m_CCND3"]
rowLastNo = rowLastNo+rowNum+1
for rowNum, val in enumerate(mRNAnames):
    tempTAs = []
    tempnAs = []
    tempKAs = []
    tempTRs = []
    tempnRs = []
    tempKRs = []
    for qq in range(numberofTARs):
        if tck50as[rowNum,qq] > 0:
            tempTAs.append(TARsRead.columns[qq])
            tempnAs.append(tcnas[rowNum,qq])
            tempKAs.append(tck50as[rowNum,qq])
        if tck50rs[rowNum,qq] > 0:
            tempTRs.append(TARsRead.columns[qq])
            tempnRs.append(tcnrs[rowNum,qq])
            tempKRs.append(tck50rs[rowNum,qq])
    j = 3
    formulaAs = []
    for pp in range(np.size(tempTAs)):
        formulaAs.append("("+tempTAs[pp]+"/"+"k"+str(rowLastNo+rowNum+1)+"_"+str(j)+")^"+"k"+str(rowLastNo+rowNum+1)+"_"+str(j+1))
        paramnames.append("k"+str(rowLastNo+rowNum+1)+"_"+str(j))
        paramvals.append(tempKAs[pp])
        paramrxns.append('vTC'+str(rowNum+1))
        paramidxs.append(j)
        paramnames.append("k"+str(rowLastNo+rowNum+1)+"_"+str(j+1))
        paramvals.append(tempnAs[pp])
        paramrxns.append('vTC'+str(rowNum+1))
        paramidxs.append(j+1)       
        j = j + 2
    if val not in mrnas2mod: # if not Cyclin D gene
        formulaAs = " + ".join(formulaAs)
    
    formulaRs = []
    for pp in range(np.size(tempTRs)):
        formulaRs.append("("+tempTRs[pp]+"/"+"k"+str(rowLastNo+rowNum+1)+"_"+str(j)+")^"+"k"+str(rowLastNo+rowNum+1)+"_"+str(j+1))
        paramnames.append("k"+str(rowLastNo+rowNum+1)+"_"+str(j))
        paramvals.append(tempKRs[pp])
        paramrxns.append('vTC'+str(rowNum+1))
        paramidxs.append(j)
        paramnames.append("k"+str(rowLastNo+rowNum+1)+"_"+str(j+1))
        paramvals.append(tempnRs[pp])
        paramrxns.append('vTC'+str(rowNum+1))
        paramidxs.append(j+1)  
        j = j + 2
    formulaRs = " + ".join(formulaRs)       

    if val in mrnas2mod: # if Cyclin D gene            
        formulaARs = formulaAs[0]+"/(1.0+"+formulaAs[0]+")"+"*"+formulaAs[1]+"/(1.0+"+formulaAs[1]+")"
    elif formulaAs and formulaRs:
        formulaARs = "("+"("+formulaAs+")"+"/(1.0+"+formulaAs+"+"+formulaRs+"))"
    elif formulaAs:
        formulaARs = "("+"("+formulaAs+")/(1.0+"+formulaAs+"))"
    elif formulaRs:
        formulaARs = "(1.0"+"/(1.0+"+formulaRs+"))"
    else:
        formulaARs = "0.0"
    j = 1
    formula = "("+"k"+str(rowLastNo+rowNum+1)+"_"+str(j)+"*"+ac_genenames[rowNum]+")+"+"("+"k"+str(rowLastNo+rowNum+1)+"_"+str(j+1)+"*"+ac_genenames[rowNum]+"*"+formulaARs+")"
    fileModel.write("  %s: %s => %s; (%s)*%s;\n" % ('vTC'+str(rowNum+1), "", val, formula, genesComp[rowNum])) # for vTC

    paramnames.append("k"+str(rowLastNo+rowNum+1)+"_"+str(j))
    paramvals.append(kTCleak[rowNum])
    paramrxns.append('vTC'+str(rowNum+1))
    paramidxs.append(1)
    paramnames.append("k"+str(rowLastNo+rowNum+1)+"_"+str(j+1))
    paramvals.append(kTCmaxs[rowNum])
    paramrxns.append('vTC'+str(rowNum+1))
    paramidxs.append(2)  

rowLastNo = rowLastNo+rowNum+1
for rowNum, val in enumerate(mRNAnames):
    formula = "("+"k"+str(rowLastNo+rowNum+1)+"*"+val+")"
    fileModel.write("  %s: %s => %s; (%s)*%s;\n" % ('vTCd'+str(rowNum+1), val, "", formula, genesComp[rowNum])) # for vTCd
    paramnames.append("k"+str(rowLastNo+rowNum+1))
    paramvals.append(kTCd[rowNum])
    paramrxns.append('vTCd'+str(rowNum+1))
    paramidxs.append(0)
    
# Export parameters for each reaction, with corresponding order within the ratelaw and its value
params_all = pd.DataFrame({'value':paramvals,'rxn':paramrxns,'idx':paramidxs},index=paramnames)
params_all.to_csv(fileParamsOut,sep='\t',header=True, index=True)

#% The Compartment initial conditions
fileModel.write("\n  # Compartment initializations:\n")
for idx in range(len(compartments)):
    fileModel.write("  %s = %.6e;\n" % (compartments[idx], np.double(volumes[idx])))
    fileModel.write("  %s has volume;\n" % (compartments[idx]))
    
# The Species initial conditions
fileModel.write("\n  # Species initializations:\n")
for idx, val in enumerate(species_sheet[1:]):
    fileModel.write("  %s = %.6e;\n" % (val[0],np.double(val[2])))
    
xgac_mpc_D = (kGac*gExp_mpc)/(kGin+kGac) #active genes initial condition
xgin_mpc_D = gExp_mpc - xgac_mpc_D
genesIC = np.concatenate((xgac_mpc_D,xgin_mpc_D))
for idx, val in enumerate(ac_genenames):
    fileModel.write("  %s = %.6e;\n" % (val,np.double(genesIC[idx]*mpc2nmcf_Vc)))
for idx, val in enumerate(ia_genenames):
    fileModel.write("  %s = %.6e;\n" % (val,np.double(genesIC[idx+len(model_genes)]*mpc2nmcf_Vc)))

# The Parameter (of reactions) initial conditions
fileModel.write("\n  # Parameter initializations:\n")
count = 0
for param in paramnames:
    fileModel.write("  %s = %.6e;\n" % (param, np.double(paramvals[count])))
    count += 1

In [ ]:
#%% Other declarations supported by Antimony
constantVars = ['Cytoplasm','Extracellular','Nucleus','Mitochondrion']
fileModel.write("\n  # Other declarations:\n")
fileModel.write("  const")
for constVar in constantVars[:-1]:
    fileModel.write("  %s," % (constVar))
#last item in row needs semicolon
fileModel.write("  %s;\n" % (constantVars[-1]))

# The Unit Definitions
fileModel.write("\n  # Unit definitions:")
fileModel.write("\n  unit time_unit = second;")
fileModel.write("\n  unit volume = litre;")
fileModel.write("\n  unit substance = 1e-9 mole;")
fileModel.write("\n  unit nM = 1e-9 mole / litre;")
fileModel.write("\n")

# End of the Antimony file, close the file
fileModel.write("\nend")
fileModel.close()

In [ ]:
#%% The Antimony file import and conversion to SBML format
if loadFile(fnameInput+".txt") == 1:
    print("Success loading antimony file")
else:
    print("Failed to load antimony file")
    exit(1)
if writeSBMLFile(fnameInput+".xml",fnameInput) == 1:
    print("Success converting antimony to SBML")
else:
    print("Failure converting antimony to SBML")

In [9]:
#%% Import the SBML file and get the model handle
sbml_reader = libsbml.SBMLReader()
sbml_doc = sbml_reader.readSBML(sbml_file)
sbml_model = sbml_doc.getModel()
# Set species annotations
for idx,row in enumerate(species_sheet[1:]):
    Annot=""
    for col in range(4,(len(row))):
        aa=str(row[col].strip())
        if aa=="nan" or aa == "":
            break
        else:
            Annot=Annot+" "+row[col]
    sbml_model.getSpecies(row[0]).setAnnotation(Annot)
# Set compartment annotations
for row in compartment_sheet[1:]:
    sbml_model.getCompartment(row[0]).setAnnotation(row[2])
# Write with the same name or use the next section instead of below lines
writer = libsbml.SBMLWriter()
writer.writeSBML(sbml_doc,sbml_file)

#%% The Model Compilation
# prepares to use interaction components to synthesize model
sys.path.insert(0, os.path.abspath(model_output_dir))
model_name = sbml_file[0:-4]
model_output_dir = model_name

sbml_reader = libsbml.SBMLReader()
sbml_doc = sbml_reader.readSBML(sbml_file)
sbml_model = sbml_doc.getModel()

# Create an SbmlImporter instance for our SBML model
sbml_importer = amici.SbmlImporter(sbml_file)
#sets important constants for model build
constantParameters = [params.getId() for params in sbml_model.getListOfParameters()]
observables = {'actp53':{'formula': 'p53ac'},'phERK':{'formula': 'ppERK'},'egfLR':{'formula': 'EE1'}}
# The actual compilation step by AMICI, takes a while to complete for large models
sbml_importer.sbml2amici(model_name,
                         model_output_dir,
                         verbose=False,
                         observables=observables,
                         constantParameters=constantParameters)